# ETL 설계 4편: SCD - 고객 주소가 바뀌면 과거 주문은 어디로 배송된 걸로 남는가

차원 데이터가 바뀌면 과거를 덮어쓸 것인가, 이력을 남길 것인가. SCD Type 1, 2, 3의 차이를 SQL로 직접 구현하고, dbt snapshot으로 프로덕션 패턴을 만든다.

📖 [블로그 원문](https://biz-agentic-ai.github.io/guides/etl-design/004-scd/)

## 환경 세팅

In [1]:
# 도구 설치 (protobuf 경고는 무시해도 됩니다 - Colab의 tensorflow와 버전 충돌이지만 실습에 영향 없음)
!pip install -q duckdb dbt-core dbt-duckdb 2>&1 | grep -v "protobuf"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Bronze 데이터 준비

1편에서 만든 Bronze 데이터를 다시 적재한다.

In [2]:
import duckdb

conn = duckdb.connect('warehouse.duckdb')

conn.execute("""
CREATE SCHEMA IF NOT EXISTS bronze;

CREATE OR REPLACE TABLE bronze.orders AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv');

CREATE OR REPLACE TABLE bronze.customers AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_customers.csv');

CREATE OR REPLACE TABLE bronze.payments AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_payments.csv');
""")

print("Bronze 적재 완료")
conn.close()

Bronze 적재 완료


## 문제 확인: 차원이 바뀌면 과거가 왜곡된다

고객 테이블에서 주소를 UPDATE한 뒤, 과거 주문을 조회해 본다.

In [3]:
conn = duckdb.connect('warehouse.duckdb')

# 차원 테이블 생성 (주소 포함)
conn.execute("""
CREATE OR REPLACE TABLE dim_customers AS
SELECT
    id AS customer_id,
    first_name,
    last_name,
    CASE
        WHEN id % 3 = 0 THEN '서울'
        WHEN id % 3 = 1 THEN '부산'
        ELSE '대전'
    END AS city
FROM bronze.customers;
""")

# customer_id = 1의 원래 주소 확인
print("=== 변경 전 ===")
display(conn.execute("""
SELECT customer_id, first_name, city
FROM dim_customers WHERE customer_id = 1
""").fetchdf())

=== 변경 전 ===


,customer_id,first_name,city
0,1,Michael,부산


In [4]:
# 주문과 조인 — 변경 전
print("=== 주문 + 고객 주소 (변경 전) ===")
display(conn.execute("""
SELECT o.id AS order_id, o.order_date, c.city
FROM bronze.orders o
JOIN dim_customers c ON o.user_id = c.customer_id
WHERE o.user_id = 1
ORDER BY o.order_date
""").fetchdf())

=== 주문 + 고객 주소 (변경 전) ===


,order_id,order_date,city
0,1,2018-01-01,부산
1,37,2018-02-10,부산


In [5]:
# Type 1: 덮어쓰기
conn.execute("""
UPDATE dim_customers
SET city = '제주'
WHERE customer_id = 1;
""")

# 주문과 조인 — 변경 후
print("=== 주문 + 고객 주소 (변경 후) ===")
display(conn.execute("""
SELECT o.id AS order_id, o.order_date, c.city
FROM bronze.orders o
JOIN dim_customers c ON o.user_id = c.customer_id
WHERE o.user_id = 1
ORDER BY o.order_date
""").fetchdf())

print("\n모든 과거 주문의 city가 '제주'로 바뀌었다. 실제로는 '부산'에서 주문한 건데.")

=== 주문 + 고객 주소 (변경 후) ===


,order_id,order_date,city
0,1,2018-01-01,제주
1,37,2018-02-10,제주



모든 과거 주문의 city가 '제주'로 바뀌었다. 실제로는 '부산'에서 주문한 건데.


## SCD Type 2 — SQL로 직접 구현

`valid_from`, `valid_to`, `is_current` 컬럼을 추가해서 이력을 관리한다.

In [6]:
# Type 2 차원 테이블 생성
conn.execute("""
CREATE OR REPLACE TABLE dim_customers_scd2 (
    dim_customers_sk INTEGER,
    customer_id INTEGER,
    first_name VARCHAR,
    last_name VARCHAR,
    city VARCHAR,
    valid_from DATE,
    valid_to DATE,
    is_current BOOLEAN
);
""")

# 초기 데이터 적재 — 모든 행이 현재 유효
conn.execute("""
INSERT INTO dim_customers_scd2
SELECT
    row_number() OVER () AS dim_customers_sk,
    id AS customer_id,
    first_name,
    last_name,
    CASE
        WHEN id % 3 = 0 THEN '서울'
        WHEN id % 3 = 1 THEN '부산'
        ELSE '대전'
    END AS city,
    DATE '2025-01-01' AS valid_from,
    DATE '9999-12-31' AS valid_to,
    true AS is_current
FROM bronze.customers;
""")

print("=== 초기 상태 (customer_id = 1) ===")
display(conn.execute("""
SELECT * FROM dim_customers_scd2 WHERE customer_id = 1
""").fetchdf())

=== 초기 상태 (customer_id = 1) ===


,dim_customers_sk,customer_id,first_name,last_name,city,valid_from,valid_to,is_current
0,1,1,Michael,P.,부산,2025-01-01,9999-12-31,True


In [7]:
# customer_id = 1이 부산에서 제주로 이사
max_sk = conn.execute("SELECT MAX(dim_customers_sk) FROM dim_customers_scd2").fetchone()[0]

# 1단계: 기존 행을 닫는다
conn.execute("""
UPDATE dim_customers_scd2
SET valid_to = DATE '2026-02-15',
    is_current = false
WHERE customer_id = 1
  AND is_current = true;
""")

# 2단계: 새 행을 추가한다
conn.execute(f"""
INSERT INTO dim_customers_scd2
VALUES ({max_sk + 1}, 1, 'Michael', 'P.', '제주',
        DATE '2026-02-15', DATE '9999-12-31', true);
""")

print("=== Type 2 적용 후 (customer_id = 1) ===")
display(conn.execute("""
SELECT * FROM dim_customers_scd2
WHERE customer_id = 1
ORDER BY valid_from
""").fetchdf())

=== Type 2 적용 후 (customer_id = 1) ===


,dim_customers_sk,customer_id,first_name,last_name,city,valid_from,valid_to,is_current
0,1,1,Michael,P.,부산,2025-01-01,2026-02-15,False
1,101,1,Michael,P.,제주,2026-02-15,9999-12-31,True


In [8]:
# 시점 조회: 주문 시점의 고객 주소로 조인
print("=== 주문 시점의 실제 주소로 조인 ===")
display(conn.execute("""
SELECT
    o.id AS order_id,
    o.order_date,
    c.city AS city_at_order_time,
    c.valid_from,
    c.valid_to
FROM bronze.orders o
JOIN dim_customers_scd2 c
  ON o.user_id = c.customer_id
 AND CAST(o.order_date AS DATE) >= c.valid_from
 AND CAST(o.order_date AS DATE) < c.valid_to
WHERE o.user_id = 1
ORDER BY o.order_date
""").fetchdf())

print("\n과거 주문은 '부산', 최근 주문은 '제주'. 시점별 실제 값이 보존된다.")

=== 주문 시점의 실제 주소로 조인 ===


,order_id,order_date,city_at_order_time,valid_from,valid_to



과거 주문은 '부산', 최근 주문은 '제주'. 시점별 실제 값이 보존된다.


## SCD Type 3 — 이전 값을 컬럼으로 남긴다

이력 깊이가 1단계면 충분할 때. 별도 컬럼에 직전 값을 저장한다.

In [9]:
# Type 3 차원 테이블
conn.execute("""
CREATE OR REPLACE TABLE dim_customers_scd3 AS
SELECT
    id AS customer_id,
    first_name,
    last_name,
    CASE
        WHEN id % 3 = 0 THEN '서울'
        WHEN id % 3 = 1 THEN '부산'
        ELSE '대전'
    END AS city,
    NULL::VARCHAR AS previous_city
FROM bronze.customers;
""")

print("=== Type 3 변경 전 ===")
display(conn.execute("SELECT * FROM dim_customers_scd3 WHERE customer_id = 1").fetchdf())

# customer_id = 1: 부산 → 제주
conn.execute("""
UPDATE dim_customers_scd3
SET previous_city = city,
    city = '제주'
WHERE customer_id = 1;
""")

print("\n=== Type 3 변경 후 ===")
display(conn.execute("SELECT * FROM dim_customers_scd3 WHERE customer_id = 1").fetchdf())

print("\n행 수는 그대로. 직전 값만 previous_city에 남는다.")

=== Type 3 변경 전 ===


,customer_id,first_name,last_name,city,previous_city
0,1,Michael,P.,부산,None



=== Type 3 변경 후 ===


,customer_id,first_name,last_name,city,previous_city
0,1,Michael,P.,제주,부산



행 수는 그대로. 직전 값만 previous_city에 남는다.


In [10]:
conn.close()

## 실습 데이터 준비

jaffle_shop raw_customers에는 주소 컬럼이 없다. dbt snapshot 시연을 위해 합성 데이터를 만든다.

In [ ]:
conn = duckdb.connect('warehouse.duckdb')

# SCD 시연용 고객 데이터: city, membership_grade, updated_at 추가
conn.execute("""
CREATE OR REPLACE TABLE bronze.customers_v2 AS
SELECT
    id AS customer_id,
    first_name,
    last_name,
    CASE
        WHEN id % 3 = 0 THEN '서울'
        WHEN id % 3 = 1 THEN '부산'
        ELSE '대전'
    END AS city,
    CASE
        WHEN id % 4 = 0 THEN 'Gold'
        WHEN id % 4 = 1 THEN 'Silver'
        WHEN id % 4 = 2 THEN 'Bronze'
        ELSE 'Standard'
    END AS membership_grade,
    TIMESTAMP '2025-01-15 09:00:00' AS updated_at
FROM read_csv_auto(
    'https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_customers.csv'
);
""")

print("=== bronze.customers_v2 ===")
display(conn.execute("SELECT * FROM bronze.customers_v2 LIMIT 5").fetchdf())

In [ ]:
# 변경분 데이터: 일부 고객이 이사했다
conn.execute("""
CREATE OR REPLACE TABLE bronze.customers_v2_updated AS
SELECT
    customer_id,
    first_name,
    last_name,
    CASE
        WHEN customer_id IN (1, 3, 5) THEN '제주'
        ELSE city
    END AS city,
    CASE
        WHEN customer_id IN (2, 4) THEN 'Gold'
        ELSE membership_grade
    END AS membership_grade,
    CASE
        WHEN customer_id IN (1, 2, 3, 4, 5) THEN TIMESTAMP '2026-02-20 14:00:00'
        ELSE updated_at
    END AS updated_at
FROM bronze.customers_v2;
""")

# 변경된 고객 확인
print("=== 변경된 고객 ===")
display(conn.execute("""
SELECT a.customer_id,
       a.city AS before_city, b.city AS after_city,
       a.membership_grade AS before_grade, b.membership_grade AS after_grade
FROM bronze.customers_v2 a
JOIN bronze.customers_v2_updated b ON a.customer_id = b.customer_id
WHERE a.city != b.city OR a.membership_grade != b.membership_grade
""").fetchdf())

conn.close()

## dbt 프로젝트 세팅

In [ ]:
import os

os.makedirs('jaffle_shop/models/staging', exist_ok=True)
os.makedirs('jaffle_shop/models/marts', exist_ok=True)
os.makedirs('jaffle_shop/snapshots', exist_ok=True)
print("디렉토리 생성 완료")

In [ ]:
%%writefile jaffle_shop/dbt_project.yml
name: 'jaffle_shop'
version: '1.0.0'
profile: 'jaffle_shop'

model-paths: ["models"]
snapshot-paths: ["snapshots"]

In [ ]:
%%writefile jaffle_shop/profiles.yml
jaffle_shop:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: /content/warehouse.duckdb

## dbt snapshot — timestamp 전략

`updated_at` 컬럼을 기준으로 변경 여부를 판단한다.

In [ ]:
%%writefile jaffle_shop/snapshots/snap_customers.sql
{% snapshot snap_customers %}

{{
    config(
        target_schema='snapshots',
        unique_key='customer_id',
        strategy='timestamp',
        updated_at='updated_at'
    )
}}

select * from bronze.customers_v2

{% endsnapshot %}

In [ ]:
# 첫 번째 snapshot 실행
!cd jaffle_shop && dbt snapshot

In [ ]:
conn = duckdb.connect('warehouse.duckdb')

# 첫 실행 결과 확인 — 모든 행이 현재 유효 (dbt_valid_to = NULL)
print("=== snapshot 첫 실행 결과 ===")
display(conn.execute("""
SELECT customer_id, city, membership_grade,
       dbt_valid_from, dbt_valid_to
FROM snapshots.snap_customers
ORDER BY customer_id
LIMIT 5
""").fetchdf())

print("\ndbt_valid_to가 전부 NULL → 현재 유효한 행")

## 변경 데이터 투입 후 재실행

In [ ]:
# 소스 테이블을 변경분으로 교체
conn.execute("""
CREATE OR REPLACE TABLE bronze.customers_v2 AS
SELECT * FROM bronze.customers_v2_updated;
""")

print("소스 데이터 변경 완료")
conn.close()

In [ ]:
# 두 번째 snapshot 실행 — 변경 감지
!cd jaffle_shop && dbt snapshot

In [ ]:
conn = duckdb.connect('warehouse.duckdb')

# 이력이 생성된 고객 확인
print("=== customer_id = 1, 3 이력 확인 ===")
display(conn.execute("""
SELECT customer_id, city, membership_grade,
       dbt_valid_from, dbt_valid_to
FROM snapshots.snap_customers
WHERE customer_id IN (1, 3)
ORDER BY customer_id, dbt_valid_from
""").fetchdf())

print("\n한 customer_id에 행이 두 개. 기존 행의 dbt_valid_to가 채워졌다.")

In [ ]:
# 변경되지 않은 고객은 여전히 한 행
print("=== 변경되지 않은 고객 (customer_id = 6) ===")
display(conn.execute("""
SELECT customer_id, city, membership_grade,
       dbt_valid_from, dbt_valid_to
FROM snapshots.snap_customers
WHERE customer_id = 6
""").fetchdf())

## check 전략

`updated_at`이 없는 소스에는 `check` 전략을 쓴다. 지정한 컬럼의 값이 바뀌었는지 직접 비교한다.

In [ ]:
%%writefile jaffle_shop/snapshots/snap_customers_check.sql
{% snapshot snap_customers_check %}

{{
    config(
        target_schema='snapshots',
        unique_key='customer_id',
        strategy='check',
        check_cols=['city', 'membership_grade']
    )
}}

select
    customer_id,
    first_name,
    last_name,
    city,
    membership_grade
from bronze.customers_v2

{% endsnapshot %}

## snapshot을 Silver 모델에서 참조한다

dbt의 `ref()`로 snapshot 결과를 Silver 모델에서 참조한다. `dbt_valid_from`/`dbt_valid_to`를 분석에 쓰기 편한 형태로 정리한다.

In [ ]:
%%writefile jaffle_shop/models/staging/stg_customers_hist.sql
with source as (
    select * from {{ ref('snap_customers') }}
),

cleaned as (
    select
        customer_id,
        first_name,
        last_name,
        city,
        membership_grade,
        dbt_valid_from AS valid_from,
        coalesce(dbt_valid_to, '9999-12-31'::timestamp) AS valid_to,
        dbt_valid_to IS NULL AS is_current
    from source
)

select * from cleaned

In [ ]:
# 기존 Silver 모델도 함께 작성
%%writefile jaffle_shop/models/staging/stg_orders.sql
with source as (
    select * from bronze.orders
),

cleaned as (
    select
        id as order_id,
        user_id as customer_id,
        cast(order_date as date) as order_date,
        status
    from source
)

select * from cleaned

In [ ]:
# Silver 모델 실행
!cd jaffle_shop && dbt run --select staging

In [ ]:
# Silver 이력 모델 확인
print("=== stg_customers_hist (customer_id = 1) ===")
display(conn.execute("""
SELECT * FROM stg_customers_hist
WHERE customer_id = 1
ORDER BY valid_from
""").fetchdf())

## Gold에서 시점 조인

주문 시점의 고객 정보로 조인한다. 이것이 SCD Type 2의 핵심 활용이다.

In [ ]:
# Gold: 주문 시점의 고객 정보로 조인
print("=== 주문 시점 기준 고객 조인 ===")
display(conn.execute("""
SELECT
    o.order_id,
    o.order_date,
    o.customer_id,
    c.city AS city_at_order,
    c.membership_grade AS grade_at_order,
    c.is_current
FROM stg_orders o
JOIN stg_customers_hist c
  ON o.customer_id = c.customer_id
 AND o.order_date >= c.valid_from
 AND o.order_date < c.valid_to
WHERE o.customer_id IN (1, 2, 3)
ORDER BY o.customer_id, o.order_date
""").fetchdf())

print("\n각 주문 시점의 실제 값으로 조인된다.")

## 정리

| 패턴 | 적용 대상 | 구현 방법 |
|------|-----------|----------|
| Type 1 (덮어쓰기) | 오타, 코드 설명, 전화번호 | 단순 UPDATE |
| Type 2 (이력 추가) | 주소, 등급, 소속 부서 | dbt snapshot (timestamp / check) |
| Type 3 (이전 값 보존) | 조직 개편 전후 비교 | previous_ 컬럼 추가 |
| 혼합 | 하나의 테이블 내 컬럼별 구분 | Type 1 + Type 2 병행 |

In [ ]:
# 정리
conn.close()